In [1]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topologyqueries,linkedlist,update_topology,spanning_tree

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123

requestedTime=1570041125



global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

gapps = GridAPPSD()
assert gapps.connected

In [2]:
# Query for ACLineSegment, Transformerend, Switches, Breakers, Reclosers, Fuses, Sectionalisers,SynchronousMachine,TopologicalNode
topologyqueries.getallqueries(gapps,model_mrid)
Line_query=topologyqueries.Line_query
XfmrDict=topologyqueries.XfmrDict
XfmrKeys=topologyqueries.XfmrKeys
SwitchDict=topologyqueries.SwitchDict
SwitchKeys=topologyqueries.SwitchKeys
DG_query=topologyqueries.DG_query
Cap_query=topologyqueries.Cap_query
Node_query=topologyqueries.Node_query

In [3]:
# Build Linknet Lists from ACLineSegment,Transformers,DGs and Nodes
[ConnNodeDict,TerminalsDict,TermList,NodeList]=linkedlist.build_linked_list(Line_query,{}, [],DG_query,Node_query)
[LVConnNodeDict,LVTerminalsDict,LVTermList,LVNodeList]=linkedlist.build_linked_list({},XfmrDict, XfmrKeys, DG_query,Node_query)

# Stash a copy of base dictionary
BaseConnDict = json.dumps(ConnNodeDict)
BaseTermDict = json.dumps(TerminalsDict)

Processed  5294 buses in  0.04273080600000001 seconds
Processed  3909 line objects in  0.02323686800000002 seconds
Processed  0 transformer objects in  2.924999999986966e-06 seconds
Processed  9 generator objects in  2.9913999999964247e-05 seconds
Processed  27 missing nodes in  0.0020346910000000884 seconds
Processed  5294 buses in  0.01273563099999997 seconds
Processed  0 line objects in  3.3629999999229554e-06 seconds
Processed  1304 transformer objects in  0.0074680710000000206 seconds
Processed  9 generator objects in  2.9222999999967136e-05 seconds
Processed  2760 missing nodes in  0.002548097000000027 seconds


In [4]:
DistAppStruct = {}
DistAppStruct['feeders'] = {}
DistAppStruct['feeders'][model_mrid] = {}
DistAppStruct['feeders'][model_mrid]['addressabe_equipment'] = []
DistAppStruct['feeders'][model_mrid]['unaddressabe_equipment'] = []
DistAppStruct['feeders'][model_mrid]['switch_areas'] = []


In [11]:
def SwitchTree(ConnNodeDict, TerminalsDict, SwitchDict, SwitchKeys):
    Tree={}
    #ProcessedNodes=[]
    TotalNodes=0
    StartTime = time.process_time()

    # Iterate through all substation transformers
    for i6 in range(len(SwitchKeys)):



        ConnNodeDict[SwitchDict[SwitchKeys[i6]]['node1']]['switch_id'] = SwitchKeys[i6]
        ConnNodeDict[SwitchDict[SwitchKeys[i6]]['node2']]['switch_id'] = SwitchKeys[i6]

        FirstNode = 1 
        LastNode = 2
        # Set switch as rootnode 
        Tree[SwitchKeys[i6]]=[SwitchDict[SwitchKeys[i6]]['node1'], SwitchDict[SwitchKeys[i6]]['node2']]

        while LastNode != FirstNode:
            NextTerm = ConnNodeDict[Tree[SwitchKeys[i6]][FirstNode]]['list']
            FirstNode = FirstNode + 1
            while NextTerm != 0:
                # Get next node and terminal for current node
                NextNode = TerminalsDict[TermList[NextTerm-1]]['far']
                NextTerm = TerminalsDict[TermList[NextTerm-1]]['next']
                node = NodeList[NextNode-1]
                not_in_tree = True

                # Check if node already in another tree
                for i7 in range(len(Tree.keys())):
                    if node in Tree[SwitchKeys[i7]]:
                        not_in_tree = False
                        break

                # Add node if not in another tree        
                if not_in_tree:       
                    if ConnNodeDict[node]['nomv']:
                        if int(ConnNodeDict[node]['nomv']) < 34000 and int(ConnNodeDict[node]['nomv']) > 1000:
                            Tree[SwitchKeys[i6]].append(NodeList[NextNode-1])
                            LastNode = LastNode + 1                       
                    else:
                        Tree[SwitchKeys[i6]].append(NodeList[NextNode-1])
                        LastNode = LastNode + 1

        NodesInTree=len(Tree[SwitchKeys[i6]])
        print("Processed topology from  ", SwitchDict[SwitchKeys[i6]]['name'], " with id ", SwitchKeys[i6], ' with ', NodesInTree, " buses")
        TotalNodes=TotalNodes+NodesInTree

    print("Processed ", len(Tree.keys()), "topology trees containing ", TotalNodes, " buses in ", time.process_time() - StartTime, "seconds")

    return Tree, ConnNodeDict


In [13]:
[SwitchTrees, SWConnNodeDict] = SwitchTree(ConnNodeDict, TerminalsDict, SwitchDict, SwitchKeys)

Processed topology from   2002200004641085_sw  with id  _2988C4A4-4F1A-4E2F-8947-E79A582DE021  with  78  buses
Processed topology from   2002200004868472_sw  with id  _1CB6F0F5-08B3-456C-9D7D-9BCC866A2FB4  with  17  buses
Processed topology from   2002200004991174_sw  with id  _674624BB-C2D9-4402-9A0B-A491066F4CDF  with  18  buses
Processed topology from   a333_48332_sw  with id  _E56BC77B-6895-4243-ABFC-EA44A3E8AAE2  with  14  buses
Processed topology from   a8611_48332_sw  with id  _AAF94161-A827-463F-B9B6-7E8CCCFC969F  with  64  buses
Processed topology from   a8645_48332_sw  with id  _EE49C3F2-1112-4C9C-B7FA-9BFFD48012FC  with  22  buses
Processed topology from   a8869_48332_sw  with id  _C2D45253-1D4D-4EEC-90D9-494DB20F3A12  with  41  buses
Processed topology from   dg1069mt1_sw  with id  _035FD479-EF61-4311-9658-7EA265034AD2  with  2  buses
Processed topology from   dg1089dies_sw  with id  _A50CDB24-727C-48A4-B658-E7C433F371F4  with  2  buses
Processed topology from   dg1089lng_s

In [ ]:
SwitchArea = {}
SwitchArea['boundary_switches'] = []
SwitchArea['addressable_equipment'] = []
SwitchArea['unaddressable_equipment'] = []
SwitchArea['connectivity_node'] = []
SwitchArea['secondary_areas'] = []

In [6]:
switch = '_1962A006-8FA7-403E-A06F-DCBFE991E771'
for i8 in range(len(Tree[switch])):
    print(ConnNodeDict[Tree[switch][i8]]['name'])

d6108141-1_int
e206209
l2897777
n1144665
m1047515
m1047513
m1047507
p901913
l2820531
196-35813
l3157708
n1137991
m4113345
l3197640


In [7]:
ConnNodeDict[Tree[switch][5]]

{'name': 'm1047513',
 'tpid': '_73DA8CAA-3B16-4288-921F-7DED28B6BF1B',
 'nomv': [],
 'ACLineSegment.name': ['ln5593236-6', 'ln5773033-1', 'ln6047577-1'],
 'ACLineSegment.mRID': ['_6E8D6D63-71D0-4C3C-807A-8E1E176450BF',
  '_CA50789D-5E44-47D7-B3D6-6E549027A1A8',
  '_BBBD4201-2CFA-4498-AB2C-94E689E61088'],
 'TransformerEnd.name': [],
 'TransformerEnd.mRID': [],
 'switch.name': [],
 'switch.mRID': [],
 'SynchronousMachine.name': [],
 'SynchronousMachine.mRID': [],
 'node': 726,
 'list': 3077}

In [8]:
len(ConnNodeDict.keys())

5294

In [9]:
len(NodeList)

5294